In [ ]:
import time
import uuid

from pymongo import MongoClient


from indexer.schema import header, group, stream, doctypes, dtypes
from p4p import Type, Value
from p4p.nt import NTMultiChannel, NTTable

# TODO: Add alarm status and severity of failed requests

class DixrRO:
    def __init__(self, config):
        self.config = config
        

    @property
    def config(self):
        return self._config

    @config.setter
    def config(self, config):
        for key in ['host', 'port', 'database']:
            if key not in config:
                raise KeyError('Required fields missing')
        self._config = config
        
    @property
    def conn(self):
        return MongoClient(host=self.config['host'],
                           port=self.config['port'])
        
    @property
    def db(self):
        return self.conn[self.config['database']]

    def query(self, doctype, **kwargs):
        # TODO: Use a generator instead of return
        cursor = self.db[doctype].find(kwargs)
        result = list(cursor)
        normed = self._normalize_results(doctype=doctype,
                                         results=result)
        return self._bson2mtch(doctype=doctype,
                               docs=normed)

    def _pv2bson(self, doc, doctype):
        """Convert from pvdata Value to python dict """
        lookup = {'i': int, 'd': float, 's': str, 'ai': int, 'ad': float, 'as': str}
        bson = {}
        for entry in dtypes[doctype]:
            # TODO: A more sophisticated way to fetch array
            bson[entry[0]] = lookup[entry[1]](doc.get(entry[0]))
        return bson

    def _bson2pv(self, doctype, doc):
        """Given a dictionary, return a pvdata structure"""
        T = Type(dtypes[doctype])
        return Value(T, doc)

    def _bson2mtch(self, doctype, docs):
        # TODO: Exception handling if wrong doc format provided
        table_template = NTTable.buildType(columns=dtypes[doctype])    
        table =  Value(table_template,{'value': docs, 
                                       'timeStamp.secondsPastEpoch': time.time(),
                                        'descriptor': doctype})
        return table
    
    def _normalize_results(self, doctype, results):
        keys = [i[0] for i in dtypes[doctype]]
        tmp_dict = {}
        for k in keys:
            tmp_dict[k] = []
        for result in results:
                for k in keys:
                    tmp_dict[k].append(result[k])
        return tmp_dict
    

class Dixr(DixrRO):
   def insert(self, doctype, contents):
       bson = self._pv2bson(doc=contents, doctype=doctype)
       self.db[doctype].insert_one(bson)

In [ ]:
config = dict(host='localhost', port=27017, database='dixrtest')

rw = Dixr(config=config)

In [ ]:
v = {'uid': str(uuid.uuid4()), 'time': time.time(), 'machine':'test_ess', 'tbd': 0}

r = DixrRO(config=config)
rw.insert(doctype='header', contents=v)

In [ ]:
x = r.query(doctype='header')

In [ ]:
print(x)

Mongo Screenshot of header to mongo document
> show dbs
admin  0.000GB
dixt2  0.000GB
local  0.000GB
> use dixt2
switched to db dixt2
> db.header.find()
{ "_id" : ObjectId("5970c6ba304e2a561d0aa10e"), "uid" : "6dc4aa55-a673-447e-b832-9b1484a66fb8", "time" : 1500563130.9218612, "machine" : "test_ess", "tbd" : 0 }
{ "_id" : ObjectId("5970c6cd304e2a561d0aa113"), "uid" : "a23d5bda-0c69-4210-89f4-30307d09de42", "time" : 1500563149.630261, "machine" : "test_ess", "tbd" : 0 }
>

In [6]:
x.tolist()

[('labels', []),
 ('value',
  [('uid',
    ['ddbdc90e-6d75-47e4-a71c-abb8bb30f000',
     'b230d17e-8c1c-41b9-aeba-9ccbb98a2b1c',
     'a2b002d4-6a30-446e-89fb-e007c21105fe',
     '06e504cf-943c-45f3-94cc-140d876681da',
     '1d8b39f0-525d-4429-acdd-9c500f1e529f',
     '2435d184-65f8-42fa-96c6-0af5942d4d61',
     '70cc1ce2-1ddc-4aa2-83ca-f526a9f64678',
     'f622b220-2e5d-477b-aacb-bd7805d67f05',
     'f3c483e0-4a96-47e0-b5e8-8b0added4e50',
     'bd7d4e0c-99a3-4608-abd4-8ece822520c7',
     'e2e41264-7343-435f-b554-a761b0022755',
     '309e7c62-b1bf-496e-8a66-a2aa5935d3d5',
     'dc381bdc-1264-4d2e-9464-d0d456e512d7',
     'a1c3960c-01ec-43a0-a712-975ca2a6dcb1',
     'b711abfb-aaa0-4669-87d2-b3892dc16a84',
     'ea46d462-9225-4963-bd67-4b6c05e97772',
     '1ba0c1c2-c728-400f-8609-c2dffeb3fc2b',
     'b98516a4-0ffe-4d49-b83d-9f4967d3373d',
     'a2bb1d4b-0054-4d78-a325-f5b08547b213']),
   ('time', array([  1.50058556e+09,   1.50058709e+09,   1.50058712e+09,
             1.50058718e+09,   

In [7]:
x.items()

[('labels', []),
 ('value',
  Value(id:structure, value.uid:['ddbdc90e-6d75-47e4-a71c-abb8bb30f000', 'b230d17e-8c1c-41b9-aeba-9ccbb98a2b1c', 'a2b002d4-6a30-446e-89fb-e007c21105fe', '06e504cf-943c-45f3-94cc-140d876681da', '1d8b39f0-525d-4429-acdd-9c500f1e529f', '2435d184-65f8-42fa-96c6-0af5942d4d61', '70cc1ce2-1ddc-4aa2-83ca-f526a9f64678', 'f622b220-2e5d-477b-aacb-bd7805d67f05', 'f3c483e0-4a96-47e0-b5e8-8b0added4e50', 'bd7d4e0c-99a3-4608-abd4-8ece822520c7', 'e2e41264-7343-435f-b554-a761b0022755', '309e7c62-b1bf-496e-8a66-a2aa5935d3d5', 'dc381bdc-1264-4d2e-9464-d0d456e512d7', 'a1c3960c-01ec-43a0-a712-975ca2a6dcb1', 'b711abfb-aaa0-4669-87d2-b3892dc16a84', 'ea46d462-9225-4963-bd67-4b6c05e97772', '1ba0c1c2-c728-400f-8609-c2dffeb3fc2b', 'b98516a4-0ffe-4d49-b83d-9f4967d3373d', 'a2bb1d4b-0054-4d78-a325-f5b08547b213'])),
 ('descriptor', 'header'),
 ('alarm', Value(id:alarm_t, alarm.severity:0)),
 ('timeStamp', Value(id:time_t, timeStamp.secondsPastEpoch:1500644845))]

In [8]:
x.value.uid

['ddbdc90e-6d75-47e4-a71c-abb8bb30f000',
 'b230d17e-8c1c-41b9-aeba-9ccbb98a2b1c',
 'a2b002d4-6a30-446e-89fb-e007c21105fe',
 '06e504cf-943c-45f3-94cc-140d876681da',
 '1d8b39f0-525d-4429-acdd-9c500f1e529f',
 '2435d184-65f8-42fa-96c6-0af5942d4d61',
 '70cc1ce2-1ddc-4aa2-83ca-f526a9f64678',
 'f622b220-2e5d-477b-aacb-bd7805d67f05',
 'f3c483e0-4a96-47e0-b5e8-8b0added4e50',
 'bd7d4e0c-99a3-4608-abd4-8ece822520c7',
 'e2e41264-7343-435f-b554-a761b0022755',
 '309e7c62-b1bf-496e-8a66-a2aa5935d3d5',
 'dc381bdc-1264-4d2e-9464-d0d456e512d7',
 'a1c3960c-01ec-43a0-a712-975ca2a6dcb1',
 'b711abfb-aaa0-4669-87d2-b3892dc16a84',
 'ea46d462-9225-4963-bd67-4b6c05e97772',
 '1ba0c1c2-c728-400f-8609-c2dffeb3fc2b',
 'b98516a4-0ffe-4d49-b83d-9f4967d3373d',
 'a2bb1d4b-0054-4d78-a325-f5b08547b213']

In [9]:
x.value.machine

['test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess',
 'test_ess']